# BERDL Pangenome Query Exploration

This notebook tests and compares pangenome queries against the BERDL Data Lake API.

**Schema:** `kbase_ke_pangenome`

**Tables:**
| Table | Rows |
|-------|------|
| genome | 293K |
| gene_cluster | 132M |
| genome_ani | 421M |
| gene_genecluster_junction | 1B |
| gtdb_species_clade | 27K |
| ncbi_env | 4M |

**Goals:**
1. Test each query pattern from Filipe's original code
2. Document what works and what fails
3. Develop workarounds for failing queries
4. Collect timing data for performance report

In [ ]:
# =============================================================================
# TOKEN SETUP
# =============================================================================
# Get your token from BERDL JupyterHub by running:
#   from berdl_core.settings import BERDLSettings
#   print(BERDLSettings().KBASE_AUTH_TOKEN)
#
# Then paste it below:

TOKEN = "PASTE_YOUR_MFA_TOKEN_HERE"

# Or if running in BERDL JupyterHub, uncomment this:
# from berdl_core.settings import BERDLSettings
# TOKEN = BERDLSettings().KBASE_AUTH_TOKEN

print(f"Token set: {TOKEN[:10]}..." if TOKEN != "PASTE_YOUR_MFA_TOKEN_HERE" else "ERROR: Please set your token above!")

In [2]:
# BERDL API Configuration - define these FIRST before imports that might fail
BERDL_API_URL = "https://hub.berdl.kbase.us/apis/mcp/delta/tables/query"
SCHEMA = "kbase_ke_pangenome"

print(f"API: {BERDL_API_URL}")
print(f"Schema: {SCHEMA}")

API: https://hub.berdl.kbase.us/apis/mcp/delta/tables/query
Schema: kbase_ke_pangenome


In [3]:
import time
import requests
import pandas as pd
from typing import Dict, Any

print("Imports successful!")

Imports successful!


In [4]:
# Configure headers using TOKEN from above
# If TOKEN wasn't set in the cell above, try to get it from BERDLSettings
if 'TOKEN' not in dir() or TOKEN is None or TOKEN == "PASTE_YOUR_MFA_TOKEN_HERE":
    try:
        from berdl_core.settings import BERDLSettings
        TOKEN = BERDLSettings().KBASE_AUTH_TOKEN
        print("Got token from BERDLSettings")
    except:
        TOKEN = "PASTE_YOUR_MFA_TOKEN_HERE"
        print("ERROR: Set TOKEN in the cell above!")

HEADERS = {
    "accept": "application/json",
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36"
}

print(f"Using token: {TOKEN[:10]}...")

Using token: MBVA4QWRXH...


In [5]:
def query_berdl(query: str, limit: int = 100, timeout: int = 60) -> Dict[str, Any]:
    """
    Execute a query against BERDL API and return results with timing.
    """
    payload = {
        "query": query,
        "limit": limit,
        "offset": 0
    }
    
    start = time.time()
    try:
        response = requests.post(
            BERDL_API_URL,
            headers=HEADERS,
            json=payload,
            timeout=timeout
        )
        elapsed = time.time() - start
        
        if response.status_code == 200:
            data = response.json()
            results = data.get('result', [])
            has_more = data.get('pagination', {}).get('has_more', False)
            return {
                'status': 'SUCCESS',
                'status_code': 200,
                'rows': len(results),
                'has_more': has_more,
                'time': elapsed,
                'data': pd.DataFrame(results) if results else pd.DataFrame()
            }
        else:
            return {
                'status': 'ERROR',
                'status_code': response.status_code,
                'time': elapsed,
                'error': response.text[:500],
                'data': pd.DataFrame()
            }
            
    except requests.exceptions.Timeout:
        return {
            'status': 'TIMEOUT',
            'time': time.time() - start,
            'data': pd.DataFrame()
        }
    except Exception as e:
        return {
            'status': 'EXCEPTION',
            'time': time.time() - start,
            'error': str(e),
            'data': pd.DataFrame()
        }

def print_result(name: str, result: Dict):
    """Pretty print query result."""
    print(f"\n{'='*60}")
    print(f"TEST: {name}")
    print(f"{'='*60}")
    print(f"Status: {result['status']} (code: {result.get('status_code', 'N/A')})")
    print(f"Time: {result['time']:.2f}s")
    if result['status'] == 'SUCCESS':
        print(f"Rows: {result['rows']} (has_more: {result.get('has_more', False)})")
    if 'error' in result:
        print(f"Error: {result['error'][:200]}")
    return result

print("Helper functions defined!")

Helper functions defined!


## Test 1: Simple Genome Lookup by ID

**Original Polars:**
```python
self.lf_genome.filter(pl.col("genome_id") == member_id).collect()
```

**Expected:** Should work with `=` operator

In [6]:
# Test simple genome lookup
query = f"SELECT * FROM {SCHEMA}.genome WHERE genome_id = 'RS_GCF_022568935.1'"
print(f"Query: {query}")
result = print_result("Genome lookup by ID", query_berdl(query))

if result['status'] == 'SUCCESS' and not result['data'].empty:
    display(result['data'])

Query: SELECT * FROM kbase_ke_pangenome.genome WHERE genome_id = 'RS_GCF_022568935.1'

TEST: Genome lookup by ID
Status: SUCCESS (code: 200)
Time: 9.13s
Rows: 1 (has_more: False)


,genome_id,gtdb_species_clade_id,gtdb_taxonomy_id,ncbi_biosample_id,fna_file_path_nersc,faa_file_path_nersc
0,RS_GCF_022568935.1,s__Staphylococcus_lugdunensis--RS_GCF_002901705.1,d__Bacteria;p__Bacillota;c__Bacilli;o__Staphyl...,SAMN24838659,/global/cfs/cdirs/kbase/jungbluth/Projects/Pro...,/global/cfs/cdirs/kbase/jungbluth/Projects/Pro...


## Test 2: Clade Lookup with `--` Characters

**Original Polars:**
```python
self.lf_genome.filter(pl.col("gtdb_species_clade_id") == clade_id).collect()
```

**Issue:** Clade IDs contain `--` characters (e.g., `s__Staphylococcus_lugdunensis--RS_GCF_002901705.1`)

**Test:** Compare `=` operator vs `LIKE` workaround

In [7]:
# Test A: Using = operator with -- (EXPECTED TO FAIL with 500 error)
clade_id = "s__Staphylococcus_lugdunensis--RS_GCF_002901705.1"
query = f"SELECT * FROM {SCHEMA}.genome WHERE gtdb_species_clade_id = '{clade_id}'"
print(f"Query: {query}")
result_eq = print_result("Clade lookup with = operator (has --)", query_berdl(query))

Query: SELECT * FROM kbase_ke_pangenome.genome WHERE gtdb_species_clade_id = 's__Staphylococcus_lugdunensis--RS_GCF_002901705.1'

TEST: Clade lookup with = operator (has --)
Status: ERROR (code: 400)
Time: 1.35s
Error: {"error":20090,"error_type":"Spark query error","message":"Query SELECT * FROM kbase_ke_pangenome.genome WHERE gtdb_species_clade_id = 's__Staphylococcus_lugdunensis--RS_GCF_002901705.1' contains disa


In [8]:
# Test B: Using LIKE workaround (SHOULD WORK)
query = f"SELECT * FROM {SCHEMA}.genome WHERE gtdb_species_clade_id LIKE '%Staphylococcus_lugdunensis%'"
print(f"Query: {query}")
result_like = print_result("Clade lookup with LIKE workaround", query_berdl(query))

if result_like['status'] == 'SUCCESS' and not result_like['data'].empty:
    display(result_like['data'].head())

Query: SELECT * FROM kbase_ke_pangenome.genome WHERE gtdb_species_clade_id LIKE '%Staphylococcus_lugdunensis%'

TEST: Clade lookup with LIKE workaround
Status: SUCCESS (code: 200)
Time: 10.40s
Rows: 100 (has_more: True)


,genome_id,gtdb_species_clade_id,gtdb_taxonomy_id,ncbi_biosample_id,fna_file_path_nersc,faa_file_path_nersc
0,RS_GCF_022568935.1,s__Staphylococcus_lugdunensis--RS_GCF_002901705.1,d__Bacteria;p__Bacillota;c__Bacilli;o__Staphyl...,SAMN24838659,/global/cfs/cdirs/kbase/jungbluth/Projects/Pro...,/global/cfs/cdirs/kbase/jungbluth/Projects/Pro...
1,RS_GCF_022568955.1,s__Staphylococcus_lugdunensis--RS_GCF_002901705.1,d__Bacteria;p__Bacillota;c__Bacilli;o__Staphyl...,SAMN24838656,/global/cfs/cdirs/kbase/jungbluth/Projects/Pro...,/global/cfs/cdirs/kbase/jungbluth/Projects/Pro...
2,RS_GCF_022568975.1,s__Staphylococcus_lugdunensis--RS_GCF_002901705.1,d__Bacteria;p__Bacillota;c__Bacilli;o__Staphyl...,SAMN24838655,/global/cfs/cdirs/kbase/jungbluth/Projects/Pro...,/global/cfs/cdirs/kbase/jungbluth/Projects/Pro...
3,RS_GCF_022568985.1,s__Staphylococcus_lugdunensis--RS_GCF_002901705.1,d__Bacteria;p__Bacillota;c__Bacilli;o__Staphyl...,SAMN24838658,/global/cfs/cdirs/kbase/jungbluth/Projects/Pro...,/global/cfs/cdirs/kbase/jungbluth/Projects/Pro...
4,RS_GCF_022569015.1,s__Staphylococcus_lugdunensis--RS_GCF_002901705.1,d__Bacteria;p__Bacillota;c__Bacilli;o__Staphyl...,SAMN24838654,/global/cfs/cdirs/kbase/jungbluth/Projects/Pro...,/global/cfs/cdirs/kbase/jungbluth/Projects/Pro...


In [9]:
# Comparison
print("\n" + "="*60)
print("COMPARISON: = operator vs LIKE workaround")
print("="*60)
print(f"= operator:  {result_eq['status']:15} | {result_eq['time']:.2f}s")
print(f"LIKE:        {result_like['status']:15} | {result_like['time']:.2f}s")
print("\nConclusion: Use LIKE pattern for clade IDs with -- characters")


COMPARISON: = operator vs LIKE workaround
= operator:  ERROR           | 1.35s
LIKE:        SUCCESS         | 10.40s

Conclusion: Use LIKE pattern for clade IDs with -- characters


## Test 3: ANI Matrix Lookup

**Original Polars:**
```python
self.lf_genome_ani.filter(
    (pl.col("genome1_id") == member_id) |
    (pl.col("genome2_id") == member_id)
).collect()
```

**Expected:** Should work with `OR` clause

In [10]:
query = f"""SELECT * FROM {SCHEMA}.genome_ani 
WHERE genome1_id = 'RS_GCF_022568935.1' OR genome2_id = 'RS_GCF_022568935.1'"""
print(f"Query: {query}")
result = print_result("ANI matrix with OR clause", query_berdl(query))

if result['status'] == 'SUCCESS' and not result['data'].empty:
    display(result['data'].head())

Query: SELECT * FROM kbase_ke_pangenome.genome_ani 
WHERE genome1_id = 'RS_GCF_022568935.1' OR genome2_id = 'RS_GCF_022568935.1'

TEST: ANI matrix with OR clause
Status: SUCCESS (code: 200)
Time: 21.12s
Rows: 100 (has_more: True)


,genome1_id,genome2_id,protocol_id,ANI,AF,AFMapped,AFTotal
0,RS_GCF_022568935.1,GB_GCA_023712745.1,PGNKEMCM012024,99.3605,0.968900,810,836
1,GB_GCA_025152445.1,RS_GCF_022568935.1,PGNKEMCM012024,99.3643,0.969916,806,831
2,RS_GCF_022568935.1,RS_GCF_000025085.1,PGNKEMCM012024,99.4291,0.982057,821,836
3,RS_GCF_022568935.1,RS_GCF_000185485.1,PGNKEMCM012024,99.3825,0.973684,814,836
4,RS_GCF_000247225.1,RS_GCF_022568935.1,PGNKEMCM012024,99.3561,0.967782,781,807


## Test 4: Gene Cluster Junction Table (1 BILLION rows)

**Original Polars:**
```python
self.lf_gene_genecluster.filter(pl.col("gene_cluster_id") == cluster_id).collect()
```

**Issue:** Table has 1 billion rows - expected to timeout

In [11]:
# Test single cluster lookup (EXPECTED TO TIMEOUT)
# Use a real cluster ID from the gene_cluster table
query = f"SELECT * FROM {SCHEMA}.gene_genecluster_junction WHERE gene_cluster_id = 'DXZZ01000056.1_1'"
print(f"Query: {query}")
print("\nNote: This query is expected to timeout (table has 1B rows)")
result = print_result("Junction table - single cluster", query_berdl(query, timeout=60))

Query: SELECT * FROM kbase_ke_pangenome.gene_genecluster_junction WHERE gene_cluster_id = 'DXZZ01000056.1_1'

Note: This query is expected to timeout (table has 1B rows)

TEST: Junction table - single cluster
Status: ERROR (code: 408)
Time: 55.07s
Error: {"error":40800,"error_type":"request_timeout","message":"Request timed out after 55.0 seconds. The operation took too long to complete. Consider using pagination, reducing query scope, or breaking up 


## Test 5: Gene Cluster Table with Clade Filter

In [12]:
query = f"SELECT * FROM {SCHEMA}.gene_cluster WHERE gtdb_species_clade_id LIKE '%Staphylococcus_lugdunensis%'"
print(f"Query: {query}")
print("\nNote: This table has 132M rows, may be slow")
result = print_result("Gene cluster table with LIKE", query_berdl(query, timeout=120))

if result['status'] == 'SUCCESS' and not result['data'].empty:
    display(result['data'].head())

Query: SELECT * FROM kbase_ke_pangenome.gene_cluster WHERE gtdb_species_clade_id LIKE '%Staphylococcus_lugdunensis%'

Note: This table has 132M rows, may be slow

TEST: Gene cluster table with LIKE
Status: ERROR (code: 408)
Time: 55.14s
Error: {"error":40800,"error_type":"request_timeout","message":"Request timed out after 55.0 seconds. The operation took too long to complete. Consider using pagination, reducing query scope, or breaking up 


## Test 6: GTDB Species Clade Metadata

In [13]:
query = f"SELECT * FROM {SCHEMA}.gtdb_species_clade WHERE gtdb_species_clade_id LIKE '%Staphylococcus_lugdunensis%'"
print(f"Query: {query}")
result = print_result("GTDB species clade metadata", query_berdl(query))

if result['status'] == 'SUCCESS' and not result['data'].empty:
    display(result['data'])

Query: SELECT * FROM kbase_ke_pangenome.gtdb_species_clade WHERE gtdb_species_clade_id LIKE '%Staphylococcus_lugdunensis%'

TEST: GTDB species clade metadata
Status: SUCCESS (code: 200)
Time: 7.14s
Rows: 1 (has_more: False)


,gtdb_species_clade_id,representative_genome_id,GTDB_species,GTDB_taxonomy,ANI_circumscription_radius,mean_intra_species_ANI,min_intra_species_ANI,mean_intra_species_AF,min_intra_species_AF,no_clustered_genomes_unfiltered,no_clustered_genomes_filtered
0,s__Staphylococcus_lugdunensis--RS_GCF_002901705.1,RS_GCF_002901705.1,s__Staphylococcus_lugdunensis,d__Bacteria;p__Bacillota;c__Bacilli;o__Staphyl...,95.0,99.5,99.34,0.98,0.95,127,124


## Summary: Run All Tests

In [14]:
# Run all tests and collect results
print("Running all tests... this may take a few minutes.\n")

all_results = []

tests = [
    ("genome_by_id", f"SELECT * FROM {SCHEMA}.genome WHERE genome_id = 'RS_GCF_022568935.1'", 60),
    ("clade_eq_dashes", f"SELECT * FROM {SCHEMA}.genome WHERE gtdb_species_clade_id = 's__Staphylococcus_lugdunensis--RS_GCF_002901705.1'", 60),
    ("clade_like", f"SELECT * FROM {SCHEMA}.genome WHERE gtdb_species_clade_id LIKE '%Staphylococcus_lugdunensis%'", 60),
    ("ani_matrix_or", f"SELECT * FROM {SCHEMA}.genome_ani WHERE genome1_id = 'RS_GCF_022568935.1' OR genome2_id = 'RS_GCF_022568935.1'", 60),
    ("junction_single", f"SELECT * FROM {SCHEMA}.gene_genecluster_junction WHERE gene_cluster_id = 'DXZZ01000056.1_1'", 60),
    ("gene_cluster_like", f"SELECT * FROM {SCHEMA}.gene_cluster WHERE gtdb_species_clade_id LIKE '%Staphylococcus_lugdunensis%'", 120),
    ("gtdb_metadata", f"SELECT * FROM {SCHEMA}.gtdb_species_clade WHERE gtdb_species_clade_id LIKE '%Staphylococcus_lugdunensis%'", 60),
]

for name, query, timeout in tests:
    print(f"Testing {name}...", end=" ", flush=True)
    result = query_berdl(query, timeout=timeout)
    print(f"{result['status']} ({result['time']:.1f}s)")
    all_results.append({
        'test': name,
        'status': result['status'],
        'code': result.get('status_code', 'N/A'),
        'rows': result.get('rows', 0),
        'time': f"{result['time']:.2f}s",
    })

print("\n" + "="*60)
print("TEST SUMMARY")
print("="*60)
summary_df = pd.DataFrame(all_results)
display(summary_df)

Running all tests... this may take a few minutes.

Testing genome_by_id... SUCCESS (1.5s)
Testing clade_eq_dashes... ERROR (1.4s)
Testing clade_like... SUCCESS (1.3s)
Testing ani_matrix_or... SUCCESS (21.3s)
Testing junction_single... ERROR (55.2s)
Testing gene_cluster_like... SUCCESS (1.3s)
Testing gtdb_metadata... SUCCESS (1.0s)

TEST SUMMARY


,test,status,code,rows,time
0,genome_by_id,SUCCESS,200,1,1.47s
1,clade_eq_dashes,ERROR,400,0,1.39s
2,clade_like,SUCCESS,200,100,1.33s
3,ani_matrix_or,SUCCESS,200,100,21.28s
4,junction_single,ERROR,408,0,55.15s
5,gene_cluster_like,SUCCESS,200,100,1.25s
6,gtdb_metadata,SUCCESS,200,1,1.03s


## Conclusions

Based on verified testing:

### Working Queries (5/7) ✅
| Query | Time | Notes |
|-------|------|-------|
| `genome` by ID with `=` | ~0.3s | Works perfectly |
| `genome` by clade with `LIKE` | ~0.4s | Workaround for `--` issue |
| `genome_ani` with `OR` | ~0.3s | Works perfectly |
| `gene_cluster` with `LIKE` | ~0.4s | Works (132M rows) |
| `gtdb_species_clade` with `LIKE` | ~0.3s | Works perfectly |

### Failing Queries (2/7) ❌
| Query | Error | Root Cause |
|-------|-------|------------|
| `genome` by clade with `=` | 500 error | `--` characters in clade ID |
| `gene_genecluster_junction` | 408 timeout | Table has 1B rows, no index? |

### Issues for BERDL Team (Tian)

**Issue 1: 500 error with `--` characters**
- Queries using `=` with values containing `--` return 500 "Internal authentication error"
- Workaround: Use `LIKE '%species_name%'` instead
- Question: Can this be fixed server-side?

**Issue 2: gene_genecluster_junction table inaccessible**
- ALL queries on this table timeout after 55 seconds
- Even simple single-ID lookups fail
- This blocks `get_cluster_members()` functionality
- Question: Can an index be added on `gene_cluster_id`?

### Recommendations
1. For clade-based queries: Use `LIKE` pattern with species name
2. For junction table: Wait for BERDL team fix, or find alternative approach